In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **Data Analysis**

In [2]:
train = pd.read_csv("/content/train.csv")
train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [3]:
train.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [4]:
print(train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


There are a total of 891 passengers with 12 features in this dataset, however some data are empty as seen in train.describe and train.head

In [5]:
print("Survived: ", train[train['Survived']==1].shape[0])
print("Deceased: ", train[train['Survived']==0].shape[0])

Survived:  342
Deceased:  549


In [6]:
print("Males survived: ", train[(train['Survived']==1) & (train['Sex']=='male')].shape[0])
print("Females survived: ", train[(train['Survived']==1) & (train['Sex']=='female')].shape[0])
print("Males deceased: ", train[(train['Survived']==0) & (train['Sex']=='male')].shape[0])
print("Females deceased: ", train[(train['Survived']==0) & (train['Sex']=='female')].shape[0])

Males survived:  109
Females survived:  233
Males deceased:  468
Females deceased:  81


A lot of females survived when compared to males, this is most likely because the majority of females are allowed in the life boats and the males couldnt swim to safety

In [7]:
print("Passengers same or below mean age survived: ", train[(train['Survived']==1) & (train['Age']<=30)].shape[0])
print("Passengers above mean age survived: ", train[(train['Survived']==1) & (train['Age']>30)].shape[0])
print("Passengers same or below mean age deceased: ", train[(train['Survived']==0) & (train['Age']<=30)].shape[0])
print("Passengers above mean age deceased: ", train[(train['Survived']==0) & (train['Age']>30)].shape[0])


Passengers same or below mean age survived:  166
Passengers above mean age survived:  124
Passengers same or below mean age deceased:  243
Passengers above mean age deceased:  181


The sum of survived and deceased are different then before, this means there are some empty data

###**Preprocessing**

Delete features unrelated to prediction

In [8]:
train.drop(columns = ['Cabin', 'Ticket', 'Name', 'Fare', 'Embarked'], axis = 1,inplace = True)
train.head()

PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch
0            1         0       3    male  22.0      1      0
1            2         1       1  female  38.0      1      0
2            3         1       3  female  26.0      0      0
3            4         1       1  female  35.0      1      0
4            5         0       3    male  35.0      0      0

Fill up empty data

In [9]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
dtype: int64

In [10]:
mean_age = train["Age"].mean()
train.fillna(mean_age,inplace = True)

In [11]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
dtype: int64

Replace male to 1 and female to 0

In [12]:
train.replace({'Sex':{'male':1,'female':0}},inplace = True)

In [13]:
train.head()

PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch
0            1         0       3    1  22.0      1      0
1            2         1       1    0  38.0      1      0
2            3         1       3    0  26.0      0      0
3            4         1       1    0  35.0      1      0
4            5         0       3    1  35.0      0      0

The same process is applied into test data

In [14]:
test = pd.read_csv("/content/test.csv")
test.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

In [15]:
test.drop(columns = ['Cabin', 'Ticket', 'Name', 'Fare', 'Embarked'], axis = 1,inplace = True)
test.head()

PassengerId  Pclass     Sex   Age  SibSp  Parch
0          892       3    male  34.5      0      0
1          893       3  female  47.0      1      0
2          894       2    male  62.0      0      0
3          895       3    male  27.0      0      0
4          896       3  female  22.0      1      1

In [16]:
test.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
dtype: int64

In [17]:
mean_age = test["Age"].mean()
test.fillna(mean_age,inplace = True)
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
dtype: int64

In [18]:
test.replace({'Sex':{'male':1,'female':0}},inplace = True)
test.head()

PassengerId  Pclass  Sex   Age  SibSp  Parch
0          892       3    1  34.5      0      0
1          893       3    0  47.0      1      0
2          894       2    1  62.0      0      0
3          895       3    1  27.0      0      0
4          896       3    0  22.0      1      1

Splitting data

In [19]:
x = train[['Pclass','Sex','Age','SibSp','Parch']]
y = train['Survived']
x_test = test[['Pclass','Sex','Age','SibSp','Parch']]

In [20]:
x

Pclass  Sex        Age  SibSp  Parch
0         3    1  22.000000      1      0
1         1    0  38.000000      1      0
2         3    0  26.000000      0      0
3         1    0  35.000000      1      0
4         3    1  35.000000      0      0
..      ...  ...        ...    ...    ...
886       2    1  27.000000      0      0
887       1    0  19.000000      0      0
888       3    0  29.699118      1      2
889       1    1  26.000000      0      0
890       3    1  32.000000      0      0

[891 rows x 5 columns]

In [21]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [22]:
x_test

Pclass  Sex       Age  SibSp  Parch
0         3    1  34.50000      0      0
1         3    0  47.00000      1      0
2         2    1  62.00000      0      0
3         3    1  27.00000      0      0
4         3    0  22.00000      1      1
..      ...  ...       ...    ...    ...
413       3    1  30.27259      0      0
414       1    0  39.00000      0      0
415       3    1  38.50000      0      0
416       3    1  30.27259      0      0
417       3    1  30.27259      1      1

[418 rows x 5 columns]

### **Model Training & Prediction**

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
model = RandomForestClassifier(
    bootstrap=True,
    max_depth = 5,
    max_features = 'auto',
    min_samples_leaf = 4,
    min_samples_split = 10,
    n_estimators = 300
    )

In [25]:
model.fit(x, y)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=5, max_features='auto', min_samples_leaf=4,
                       min_samples_split=10, n_estimators=300)

In [26]:
y_pred = pd.read_csv("/content/gender_submission.csv")
y_pred

PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]

In [27]:
x_pred = model.predict(x_test)

### **Model Evaluation**

Accuracy score

In [28]:
from sklearn import metrics
metrics.accuracy_score(x_pred, y_pred['Survived'])

0.9401913875598086

Precision score

In [29]:
metrics.precision_score(x_pred, y_pred['Survived'])

0.8947368421052632

Recall score

In [30]:
metrics.recall_score(x_pred, y_pred['Survived'])

0.9379310344827586

f1 score

In [31]:
metrics.f1_score(x_pred, y_pred['Survived'])

0.9158249158249158

### **Making Submission**

In [32]:
submit_data = {'PassengerId': test.PassengerId, 'Survived': x_pred}
df = pd.DataFrame(submit_data)
df['PassengerId'].sort_values()
df.to_csv('predictions.csv', index = False)
df_disp = pd.read_csv('predictions.csv')
print(df_disp)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
